In [1]:
from typing import List, Optional, Generator
import pandas as pd
import numpy as np
import sqlite3
import re
import io
import math
import collections
import itertools
import functools
import random
import string
import tqdm
import bisect
import heapq

conn = sqlite3.connect(":memory:")

def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

def read_lc_df(s: str, dtypes: dict[str, str]=dict()) -> pd.DataFrame:
    temp = pd.read_csv(io.StringIO(s), sep="|", skiprows=2)
    temp = temp.iloc[1:-1, 1:-1]
    temp.columns = temp.columns.map(str.strip)
    temp = temp.map(lambda x: x if type(x) != str else None if x.strip() == 'null' else x.strip())
    temp = temp.astype(dtypes)
    return temp

conn.create_function("REGEXP", 2, regexp)

#### Helper for Binary tree problems

In [2]:
class BinaryTreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def to_list(self):
        to_visit = [self]
        visited = []
        while len(to_visit) > 0:
            curr = to_visit.pop(0)
            if curr:
                to_visit.append(curr.left)
                to_visit.append(curr.right)
                visited.append(curr.val)
            else:
                visited.append(curr)

        while visited and not visited[-1]:
            visited.pop()

        return visited

    def __str__(self):
        return str(self.val)

    @staticmethod
    def from_array(nums: list[int|None]):
        '''Create a Tree from a list of nums. Returns the root node.'''
        if len(nums) == 0:
            return None
        elif len(nums) == 1:
            return BinaryTreeNode(nums[0])
        else:
            forest = [BinaryTreeNode(nums[0])]
            parent_idx = -1
            for i in range(1, len(nums)):

                curr = None
                if nums[i] is not None:
                    curr = BinaryTreeNode(nums[i])
                    forest.append(curr)

                if i % 2 == 1:
                    parent_idx += 1
                    forest[parent_idx].left = curr
                else:
                    forest[parent_idx].right = curr

        return forest[0]

#### Helper for Singly Linked lists

In [3]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

    def __str__(self):
        return str(self.val)

    @staticmethod
    def to_singly_linked_list(nums: list[int]):
        root = prev = None
        for n in nums:
            curr = ListNode(n)
            # Init once
            if not root:
                root = curr
            if prev:
                prev.next = curr
            prev = curr

        return root

    def to_list(self) -> list[int]:
        result = []
        curr = self
        while curr:
            result.append(curr.val)
            curr = curr.next
        return result

Codeforces Practice

In [4]:
def pair_of_numbers(N: int, nums: list[int]) -> tuple[int, int, list[int]]:
    """
    Starting at index, check how far right you can go. Repeat same step on the leftwards direction.
    Combine the left and right to get the actual window that index i can cover (where nums[i] is the gcd and divides all)

    Problem occurs where there are duplicates where we skip those numbers since it was already covered in the window. Backward
    pass travels in the opp direction and we might end up missing the proper range.

    For eg:   2, 3, 1, 1, 3
    Forward:  0  0  3  0  0
    Backward: 0  0  0  4  0

    We could choose to ignore duplicates but that would miss nonoverlapping ranges
    We could try merging intervals but it wouldn't be trivial and we would need to merge only cases where the values match (need to lookback the entire length making it N^2)

    Simpler approach is to store the duplicates into an array and fill it in when we update i.
    """

    # Forward pass
    forward: dict[int, int] = {i: i for i in range(N)}
    i = j = 0
    match: list[int] = []
    while j <= N:
        if j < N and nums[j] % nums[i] == 0:
            if nums[j] == nums[i]:
                match.append(j)
            forward[i], j = j, j + 1
        else:
            for k in match:
                forward[k] = forward[i]
            i, j, match = j, j + 1, []

    # Backward pass
    backward: dict[int, int] = {i: i for i in range(N)}
    i = j = N - 1
    match = []
    while j >= -1:
        if j >= 0 and nums[j] % nums[i] == 0:
            if nums[j] == nums[i]:
                match.append(j)
            backward[i], j = j, j - 1
        else:
            for k in match:
                backward[k] = backward[i]
            i, j, match = j, j - 1, []

    # Merge both passes and store to a map seperated by distance
    valid_pairs: collections.defaultdict[int, set[int]] = collections.defaultdict(set)
    max_: int = 0
    for i in range(N):
        diff = forward[i] - backward[i]
        valid_pairs[diff].add(backward[i] + 1)
        max_ = max(max_, diff)

    return len(valid_pairs[max_]), max_, sorted(valid_pairs[max_])

# Testing the solution
assert pair_of_numbers(9, [2,4,3,6,9,8,10,2,4]) == (1, 3, [6])
assert pair_of_numbers(5, [4,6,9,3,6]) == (1, 3, [2])
assert pair_of_numbers(5, [1,3,5,7,9]) == (1, 4, [1])
assert pair_of_numbers(5, [2,3,5,7,11]) == (5, 0, [1,2,3,4,5])
assert pair_of_numbers(5, [1,2,3,2,1]) == (1, 4, [1])
assert pair_of_numbers(5, [2,4,1,1,2]) == (1, 4, [1])

# Good substrings - 271D
class Trie:
    def __init__(self, end: bool = False) -> None:
        self.nodes: dict[str, 'Trie'] = dict()
        self.end = end

    def __getitem__(self, ch: str) -> 'Trie':
        self.nodes[ch] = self.nodes.get(ch, Trie())
        return self.nodes[ch]

def good_substrings(S: str, good_chars_: str, K: int) -> int:
    N, count = len(S), 0
    good_chars: set[str] = {chr(ord('a') + i) for i in range(26) if good_chars_[i] == '1'}
    root: Trie = Trie()
    for i in range(N):
        k_, curr = K, root
        for j in range(i, N):
            if S[j] not in good_chars:
                k_ -= 1
            if k_ >= 0:
                curr = curr[S[j]]
                count += not curr.end
                curr.end = True
            else:
                break

    return count

# Testing the solution
assert good_substrings("ababab", "01000000000000000000000000", 1) == 5
assert good_substrings("acbacbacaa", "00000000000000000000000000", 2) == 8

LC POTD

In [5]:
# https://leetcode.com/problems/create-binary-tree-from-descriptions/submissions/1321920493/?envType=daily-question&envId=2024-07-15
def createBinaryTree(descriptions: list[list[int]]) -> BinaryTreeNode:
    """Node without parent is the root"""
    # Create the binary tree
    nodes_dict: dict[int, BinaryTreeNode] = dict()
    parents: set[int] = set()
    children: set[int] = set()
    for parent, child, direction in descriptions:
        p_node, c_node = nodes_dict.get(parent, BinaryTreeNode(parent)), nodes_dict.get(child, BinaryTreeNode(child))
        nodes_dict[parent], nodes_dict[child] = p_node, c_node

        if direction == 1:
            p_node.left = c_node
        else:
            p_node.right = c_node

        if parent not in children:
            parents.add(parent)
        if child in parents:
            parents.remove(child)
        children.add(child)

    assert len(parents) == 1
    return nodes_dict[parents.pop()]

# Testing the solution
assert createBinaryTree([[20,15,1],[20,17,0],[50,20,1],[50,80,0],[80,19,1]]).to_list() == [50,20,80,15,17,19]
assert createBinaryTree([[1,2,1],[2,3,0],[3,4,1]]).to_list() == [1,2,None,None,3,4]

KMP Algorithm

In [6]:
def create_lps_array(pattern: str) -> list[int]:
    N = len(pattern)
    pi: list[int] = [0] * N
    j = 0
    for i in range(1, N):
        while pattern[j] != pattern[i] and j > 0:
            j = pi[j - 1]
        if pattern[i] == pattern[j]:
            j += 1
        pi[i] = j

    return pi

# Testing the solution
assert create_lps_array("abcdabc") == [0, 0, 0, 0, 1, 2, 3]

Codeforces Practice

In [7]:
def longest_prefix_suffix_palindome(S: str) -> str:
    def KMP(pattern: str) -> list[int]:
        pattern_length = len(pattern)
        pi: list[int] = [0] * pattern_length
        j = 0
        for i in range(1, pattern_length):
            while pattern[i] != pattern[j] and j > 0:
                j = pi[j - 1]
            if pattern[i] == pattern[j]:
                j += 1
            pi[i] = j

        return pi

    N = len(S)
    i, j = 0, N - 1

    # Find greatest k such that S[0..k] + S[N - k - 1.. N - 1] is a palindrome
    k = 0
    for k in range(N // 2):
        if S[k] != S[N - k - 1]:
            break

    # Find longest prefix-suffix of S[k..N-k-1]
    middle = S[k: N - k]
    lps = middle[:KMP(middle + '#' + middle[::-1])[-1]]

    # Find longest prefix-suffix of reversed(S[k..N-k-1])
    middle_reversed = middle[::-1]
    lps_reversed = middle_reversed[:KMP(middle_reversed + '#' + middle_reversed[::-1])[-1]]

    return S[:k] + (lps if len(lps) > len(lps_reversed) else lps_reversed[::-1]) + S[N - k:]

# Testing the solution
assert len(longest_prefix_suffix_palindome("abcdfdcecba")) == len("abcdfdcba")
assert len(longest_prefix_suffix_palindome("abbaxyzyx")) == len("xyzyx")
assert len(longest_prefix_suffix_palindome("acbba")) == len("abba")
assert len(longest_prefix_suffix_palindome("a")) == len("a")
assert len(longest_prefix_suffix_palindome("codeforces")) == len("c")

#### LC Practice - Strings - KMP

In [8]:
# LC: https://leetcode.com/problems/find-the-index-of-the-first-occurrence-in-a-string/
def strStr(haystack: str, needle: str) -> int:
    def create_lps(pattern: str, M: int) -> list[int]:
        j = 0
        lps = [0] * M
        for i in range(1, M):
            while pattern[i] != pattern[j] and j > 0:
                j = lps[j - 1]
            if pattern[i] == pattern[j]:
                j += 1
            lps[i] = j

        return lps

    N, M = len(haystack), len(needle)
    lps = create_lps(needle, M)
    j = 0
    for i in range(N):
        while haystack[i] != needle[j] and j > 0:
            j = lps[j - 1]
        if haystack[i] == needle[j]:
            j += 1
        if j >= M:
            return i - j + 1

    return -1

# Testing the solution
assert strStr("asabadzabababababaaababaabba", "abababab") == 7

In [9]:
# https://leetcode.com/problems/repeated-substring-pattern/
def repeatedSubstringPattern(S: str) -> bool:
    def create_lps(pattern: str) -> list[int]:
        pattern_length, j = len(pattern), 0
        lps: list[int] = [0] * pattern_length
        for i in range(1, pattern_length):
            while pattern[i] != pattern[j] and j > 0:
                j = lps[j - 1]
            if pattern[i] == pattern[j]:
                j += 1
            lps[i] = j

        return lps

    # Create LPS, find the longest repeating pattern length
    # Ensure that the pattern repeats itself
    N = len(S)
    lps = create_lps(S)
    repeat_str_length = N - lps[-1]

    for i in range(N):
        if S[i % repeat_str_length] != S[i]:
            return False
    return (repeat_str_length < N) and (N % repeat_str_length == 0)

# Testing the solution
assert repeatedSubstringPattern("acbaaacbaa") == repeatedSubstringPattern("aaabbaaabb") == True
assert repeatedSubstringPattern("abcabcabc") == repeatedSubstringPattern("abaababaab") == True
assert repeatedSubstringPattern("abcabcabz") == repeatedSubstringPattern("aba") == False